In [1]:
import pickle
import pandas as pd

with open('df1.pickle','rb') as read_file:
    df1 = pickle.load(read_file)

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import json
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold
import numpy as np
from matplotlib import pyplot as plt
import sklearn
from sklearn.linear_model import HuberRegressor, LinearRegression
import warnings
warnings.filterwarnings("ignore")

### Model Training

Split the data to training/validation and testing:

Splitting to training/testing accurately measuring how well our model generalizes to new data. 

Further split training to to 10-fold training/validation is used to select the model that performs best.

Question: If we do not remove missing, it will show error message "Input contains NaN, infinity or a value too large for dtype('float64')". So removing missing is a must before training model!!

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 10 fold cross validation
y = df1['total_price_log']
# X = df1[['parking_scaled','# of beds_scaled','# of baths_scaled','sqft_scaled','Type_Apartment','Type_Condo','Type_Multi Family','Type_Single Family','City_SAN FRANCISCO','City_SAN MATEO','City_DALY CITY','City_PACIFICA']]
X = df1[['parking','# of beds','# of baths','sqft','Type_Apartment','Type_Condo','Type_Multi Family','Type_Single Family','City_SAN FRANCISCO','City_SAN MATEO','City_DALY CITY','City_PACIFICA']]
X = sm.add_constant(X)

# hold out 20% of the data for final testing
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=42)

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

#run the CV
kf = KFold(n_splits=10, shuffle=True, random_state = 42)
cv_lm_r2s, cv_lm_reg_r2s, cv_lm_poly_r2s, cv_huber_r2s  = [], [], [], [] #collect the validation results for both models


### Comparison between different error metrics:

**1. Mean_squared_error (MSE):** It's the average of squared difference between prediction and actual observation

**2. Mean_absolute_error (MAE):** Measures the average magnitude of the errors in a set of predictions, without considering their direction. All individual difference have equal weight

**3. Root_mean_squared_error (RMSE):** It's the square root of the average of squared differences between prediction and actual observation

*Difference between MAE and MSE:*

a. Because MSE square the differene before they are averaged, the MSE gives a relatively high weight to large errors. **This means the RMSE should be more useful when large errors are particularly undesirable.** (MAE is more robust to outliers, MSE is more useful if we are concerned about larger errors)
b. MAE is steady and RMSE increases as the variance associated with the frequency distribution of error magnitudes also increases
c. RMSE has a tendency to be increasingly larger than MAE as the test sample size increases
d. From an interpretation standpoint, MAE is clearly the winner. RMSE does not describe average error alone and has other implications that are more difficult to tease out and understand
e. On the other hand, one distinct advantage of RMSE over MAE is that RMSE avoids the use of taking the absolute value, which is undesirable in many mathematic calculations. MSE has nice mathematical properties which makes it easier to compute using gradient descent. MAE requires more complicated tools such as linear programming to compute the gradient.

*Similarity between MAE and MSE:*

a. Both express average model prediction error
b. Both range from 0 to infinite(+) and are indifferent to the direction of errors



#### Apply linear regression (OLS)

In [4]:
for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    # simple linear regression
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    # applying with or without standardized features to the linear regression does not make much difference !!???
    cv_lm_r2s.append(np.sqrt(mean_squared_error(y_val, lm.predict(X_val))))
    
print('Simple regression scores: ', cv_lm_r2s,'\n')

print(f'Simple mean cv mean_squared_error: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')


Simple regression scores:  [0.20667816856133708, 0.21936239455427214, 0.24352477407051412, 0.20326828378264827, 0.19303707021300082, 0.24865290698519293, 0.2570076158258979, 0.23355328937833633, 0.23844428306001184, 0.26636925522395705] 

Simple mean cv mean_squared_error: 0.231 +- 0.023


#### Apply Ridge/Lasso regularization

In Regularized Linear Regression, there is an additional component of the cost function that penalizes the "size" of the coefficients.

Why penalize a coefficient? At the simplest level, it forces a variable to be "worth it" in order to have a particularly extreme coefficient or even one that's greater than zero. The intuition is that it is a "simpler model" to have smaller coefficients (in absolute value) than larger ones - regularization means that the coefficients are constrained to lie within a narrower region, making model coefficients more stable and less extreme.

**When using regularization, we must standardize** the data so that all features are on the same scale (we subtract the mean of each column and divide by the standard deviation, giving us features with mean 0 and std 1). Since this scaling is part of our model, we need to scale using the training set feature distributions and apply the same scaling to validation and test without refitting the scaler. 

**LASSO**:
* _Pro_: great for trimming features and focusing interpretation on a few key ones
* _Con_: risk of discarding features that are actually useful

**Ridge**:
* _Pro_: great for smoothly handling multicollinearity (in case when multicollinearity exists, although beta_hat is unbiased estimator(under OLS), the variance of beta_hat is very large. Ridge works well on stablize variance of beta_hat), very nice when working with sparse features 
* _Con_: will never fully discard (collinear) features

If the mapping from features to target truly depends on only a few key features, LASSO should outperform. If instead the target actually depends on many features (even if only a little dependent), Ridge should work better.  

In [5]:
# Tuning regularization strength

from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

for i,curr_alpha in enumerate(alphalist):

    # note the use of a new sklearn utility: Pipeline to pack
    # multiple modeling steps into one fitting process 
    steps = [('standardize', StandardScaler()), 
             ('lasso', Lasso(alpha = curr_alpha))]

    pipe = Pipeline(steps)
    pipe.fit(X_train, y_train)
    
    val_set_pred = pipe.predict(X_val)
    err_vec_val[i] = mae(y_val, val_set_pred)

In [6]:
#plot the curve of validation error as alpha changes

# plt.plot(np.log10(alphalist), err_vec_val)

In [7]:
## This is the value of alpha that gave us the lowest error
alphalist[np.argmin(err_vec_val)]

0.01

In [8]:
cv_lasso_r2s = []

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    lasso_model = Lasso(alpha = 0.001) # this is a VERY HIGH regularization strength!, wouldn't usually be used

    #ridge with feature scaling, which standardize features by removing the mean and scaling to unit variance

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    lasso_model.fit(X_train_scaled, y_train)

    cv_lasso_r2s.append(np.sqrt(mean_squared_error(y_val, lasso_model.predict(X_val_scaled))))
    
print('Lasso scores: ', cv_lasso_r2s, '\n')

print(f'Lasso mean mean_squared_error: {np.mean(cv_lasso_r2s):.3f} +- {np.std(cv_lasso_r2s):.3f}')

Lasso scores:  [0.2069487081072591, 0.2194164015513407, 0.24309814140551478, 0.20312255562087939, 0.19261595392211753, 0.2482366180563881, 0.25666206625186294, 0.23356558380867815, 0.23837985996902597, 0.2664925645093741] 

Lasso mean mean_squared_error: 0.231 +- 0.023


In [9]:
lasso_predict_test = lasso_model.predict(X_test_scaled) 
# lasso_predict_all = lasso_model.predict(X_train) 

In [10]:
for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    lm_reg = Ridge(alpha=0.001)

    #ridge with feature scaling, which standardize features by removing the mean and scaling to unit variance

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    lm_reg.fit(X_train_scaled, y_train)

    cv_lm_reg_r2s.append(np.sqrt(mean_squared_error(y_val, lm_reg.predict(X_val_scaled))))
    
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Ridge mean mean_squared_error: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

Ridge scores:  [0.2066781738027288, 0.21936242069832546, 0.2435247611868896, 0.20326829729272014, 0.19303701835813672, 0.24865285177135282, 0.2570076273990186, 0.23355330156625706, 0.23844428968196812, 0.2663692766084271] 

Ridge mean mean_squared_error: 0.231 +- 0.023


#### Apply Huber Loss Function

Huber Loss Function takes the best properties of the L1 and L2 loss by being convex close to the target and steep for extreme values, usually are outliers.

In [11]:
from sklearn.linear_model import HuberRegressor, LinearRegression

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #ridge with feature scaling, which standardize features by removing the mean and scaling to unit variance

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
   
    huber = HuberRegressor().fit(X_train_scaled, y_train)
    # huber.score(X, y) 

    cv_huber_r2s.append(np.sqrt(mean_squared_error(y_val, huber.predict(X_val_scaled))))
    
print('Simple regression scores: ', cv_huber_r2s,'\n')

print(f'Simple mean mean_squared_error: {np.mean(cv_huber_r2s):.3f} +- {np.std(cv_huber_r2s):.3f}')

Simple regression scores:  [0.2041128394847529, 0.22214770117492688, 0.24616682748367688, 0.20655030608436648, 0.19421253880622802, 0.25037617534160816, 0.25908143006856615, 0.23336938853743175, 0.2383819642399312, 0.26701908530573887] 

Simple mean mean_squared_error: 0.232 +- 0.023


#### Apply Polynomial regression 

In [12]:
for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    poly = PolynomialFeatures(degree=3) 

    #polynomial regression
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)

    lm_poly = LinearRegression()
    lm_poly.fit(X_train_poly, y_train)
    cv_lm_poly_r2s.append(np.sqrt(mean_squared_error(y_val, lm_poly.predict(X_val_poly))))

print('Poly regression scores: ', cv_lm_poly_r2s)

print(f'Poly mean cv r^2: {np.mean(cv_lm_poly_r2s):.3f} +- {np.std(cv_lm_poly_r2s):.3f}')

Poly regression scores:  [0.31024140215102775, 0.7261997995031676, 1.760338315461797, 0.3858166716222978, 1.1959924164942164, 0.28479470554920805, 0.3458231004237437, 0.7815608430366922, 1.1591506888283656, 0.37900170717572806]
Poly mean cv r^2: 0.733 +- 0.472


### Final Model Selection

The results above shows different regression has similar MSE. The lasso model appears to be both better on average and has less varying results.

Since k-fold is more reliable than a single validation set, we select the linear regression(OLS) model.